##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Búsqueda semántica con vecinos más cercanos aproximados e incorporaciones de texto


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/tf2_semantic_approximate_nearest_neighbors"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/tf2_semantic_approximate_nearest_neighbors.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/tf2_semantic_approximate_nearest_neighbors.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a> </td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/hub/tutorials/tf2_semantic_approximate_nearest_neighbors.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
  <td>     <a href="https://tfhub.dev/google/nnlm-en-dim128/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelos de TF Hub</a> </td>
</table>

En este tutorial se ilustra cómo generar incorporaciones desde un modelo de [TensorFlow Hub](https://tfhub.dev) (TF-Hub) según los datos de entrada y crear un índice de vecinos más cercanos aproximados (ANN, por sus siglas en inglés) con las incorporaciones extraídas. Luego, el índice se puede usar para la coincidencia y recuperación de similitudes en tiempo real.

Cuando se trata de un gran corpus de datos, no es eficiente realizar una coincidencia exacta escaneando todo el repositorio para encontrar los elementos más similares a una consulta determinada en tiempo real. Por lo tanto, usamos un algoritmo de coincidencia de similitud aproximada que nos permite intercambiar un poco de la precisión en la búsqueda de coincidencias exactas del vecino más cercano por un aumento significativo en la velocidad.

En este tutorial, mostramos un ejemplo de búsqueda de texto en tiempo real sobre un corpus de titulares de noticias para encontrar los titulares más similares a una consulta. A diferencia de la búsqueda de palabras clave, aquí se captura la similitud semántica codificada en el texto incorporado.

Los pasos de este tutorial son:

1. Descargar datos de muestra.
2. Generar incorporaciones para los datos con un modelo de TF-Hub
3. Generar un índice ANN para las incorporaciones
4. Usar el índice para comparar similitudes

Usamos [Apache Beam](https://beam.apache.org/documentation/programming-guide/) para generar las incorporaciones del modelo de TF-Hub. También usamos la biblioteca [ANNOY](https://github.com/spotify/annoy) de Spotify para crear el índice de vecino más cercano apróximado.

### Más modelos

Para modelos que tienen la misma arquitectura pero están entrenados en un lenguaje diferente, consulte [esta](https://tfhub.dev/google/collections/nnlm/1) colección. [Aquí](https://tfhub.dev/s?module-type=text-embedding) puede encontrar todas las incorporaciones de texto que se hospedan actualmente en [tfhub.dev](https://tfhub.dev/). 

## Preparación

Instalar las bibliotecas necesarias.

In [ ]:
!pip install apache_beam
!pip install 'scikit_learn~=0.23.0'  # For gaussian_random_matrix.
!pip install annoy

Instalar las bibliotecas necesarias

In [ ]:
import os
import sys
import pickle
from collections import namedtuple
from datetime import datetime
import numpy as np
import apache_beam as beam
from apache_beam.transforms import util
import tensorflow as tf
import tensorflow_hub as hub
import annoy
from sklearn.random_projection import gaussian_random_matrix

In [ ]:
print('TF version: {}'.format(tf.__version__))
print('TF-Hub version: {}'.format(hub.__version__))
print('Apache Beam version: {}'.format(beam.__version__))

## 1. Descargar datos de muestra

El conjunto de datos [Un millón de titulares de noticias](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/SYBGZL#) contiene titulares de noticias publicados durante un período de 15 años procedentes de la prestigiosa Australian Broadcasting Corp. (ABC). Este conjunto de datos de noticias tiene un registro histórico resumido de eventos notables en el mundo desde principios de 2003 hasta finales de 2017 con un enfoque más local en Australia.

**Formato**: datos en dos columnas y separados por tabulaciones: 1) fecha de publicación y 2) texto del título. Sólo nos interesa el texto del título.


In [ ]:
!wget 'https://dataverse.harvard.edu/api/access/datafile/3450625?format=tab&gbrecs=true' -O raw.tsv
!wc -l raw.tsv
!head raw.tsv

Para simplificar, solo mantenemos el texto del título y eliminamos la fecha de publicación.

In [ ]:
!rm -r corpus
!mkdir corpus

with open('corpus/text.txt', 'w') as out_file:
  with open('raw.tsv', 'r') as in_file:
    for line in in_file:
      headline = line.split('\t')[1].strip().strip('"')
      out_file.write(headline+"\n")

In [ ]:
!tail corpus/text.txt

## 2. Generar incorporaciones para los datos.

En este tutorial, usamos el [Modelo de lenguaje de redes neuronales](https://tfhub.dev/google/nnlm-en-dim128/2) (NNLM por sus siglas en inglés) para generar incorporaciones para los datos de los titulares. Las incorporaciones de oraciones se pueden usar fácilmente para calcular la similitud de significado a nivel de oración. Ejecutamos el proceso de generación de incorporaciones con Apache Beam.

### Método de extracción de incorporaciones

In [ ]:
embed_fn = None

def generate_embeddings(text, model_url, random_projection_matrix=None):
  # Beam will run this function in different processes that need to
  # import hub and load embed_fn (if not previously loaded)
  global embed_fn
  if embed_fn is None:
    embed_fn = hub.load(model_url)
  embedding = embed_fn(text).numpy()
  if random_projection_matrix is not None:
    embedding = embedding.dot(random_projection_matrix)
  return text, embedding


### Convertir a método tf.Example

In [ ]:
def to_tf_example(entries):
  examples = []

  text_list, embedding_list = entries
  for i in range(len(text_list)):
    text = text_list[i]
    embedding = embedding_list[i]

    features = {
        'text': tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[text.encode('utf-8')])),
        'embedding': tf.train.Feature(
            float_list=tf.train.FloatList(value=embedding.tolist()))
    }
  
    example = tf.train.Example(
        features=tf.train.Features(
            feature=features)).SerializeToString(deterministic=True)
  
    examples.append(example)
  
  return examples

### Canalización de Beam

In [ ]:
def run_hub2emb(args):
  '''Runs the embedding generation pipeline'''

  options = beam.options.pipeline_options.PipelineOptions(**args)
  args = namedtuple("options", args.keys())(*args.values())

  with beam.Pipeline(args.runner, options=options) as pipeline:
    (
        pipeline
        | 'Read sentences from files' >> beam.io.ReadFromText(
            file_pattern=args.data_dir)
        | 'Batch elements' >> util.BatchElements(
            min_batch_size=args.batch_size, max_batch_size=args.batch_size)
        | 'Generate embeddings' >> beam.Map(
            generate_embeddings, args.model_url, args.random_projection_matrix)
        | 'Encode to tf example' >> beam.FlatMap(to_tf_example)
        | 'Write to TFRecords files' >> beam.io.WriteToTFRecord(
            file_path_prefix='{}/emb'.format(args.output_dir),
            file_name_suffix='.tfrecords')
    )

### Generar una matriz de peso de proyección aleatoria

[La proyección aleatoria](https://en.wikipedia.org/wiki/Random_projection) es una técnica simple pero poderosa que se usa para reducir la dimensionalidad de un conjunto de puntos que se encuentran en el espacio euclidiano. Para obtener información teórica, consulte el [lema de Johnson-Lindenstrauss](https://en.wikipedia.org/wiki/Johnson%E2%80%93Lindenstrauss_lemma).

Reducir la dimensionalidad de las incorporaciones con proyección aleatoria significa menos tiempo necesario para construir y consultar el índice ANN.

En este tutorial usamos la [proyección aleatoria gaussiana](https://en.wikipedia.org/wiki/Random_projection#Gaussian_random_projection) de la biblioteca [Scikit-learn](https://scikit-learn.org/stable/modules/random_projection.html#gaussian-random-projection).

In [ ]:
def generate_random_projection_weights(original_dim, projected_dim):
  random_projection_matrix = None
  random_projection_matrix = gaussian_random_matrix(
      n_components=projected_dim, n_features=original_dim).T
  print("A Gaussian random weight matrix was creates with shape of {}".format(random_projection_matrix.shape))
  print('Storing random projection matrix to disk...')
  with open('random_projection_matrix', 'wb') as handle:
    pickle.dump(random_projection_matrix, 
                handle, protocol=pickle.HIGHEST_PROTOCOL)
        
  return random_projection_matrix

### Establecer parámetros

Si desea crear un índice con el espacio de incorporación original sin proyección aleatoria, establezca el parámetro `projected_dim` en `None`. Tenga en cuenta que esto ralentizará el paso de indexación para incorporaciones de alta dimensión.

In [ ]:
model_url = 'https://tfhub.dev/google/nnlm-en-dim128/2' #@param {type:"string"}
projected_dim = 64  #@param {type:"number"}

### Ejecutar canalización

In [ ]:
import tempfile

output_dir = tempfile.mkdtemp()
original_dim = hub.load(model_url)(['']).shape[1]
random_projection_matrix = None

if projected_dim:
  random_projection_matrix = generate_random_projection_weights(
      original_dim, projected_dim)

args = {
    'job_name': 'hub2emb-{}'.format(datetime.utcnow().strftime('%y%m%d-%H%M%S')),
    'runner': 'DirectRunner',
    'batch_size': 1024,
    'data_dir': 'corpus/*.txt',
    'output_dir': output_dir,
    'model_url': model_url,
    'random_projection_matrix': random_projection_matrix,
}

print("Pipeline args are set.")
args

In [ ]:
print("Running pipeline...")
%time run_hub2emb(args)
print("Pipeline is done.")

In [ ]:
!ls {output_dir}

Lea algunas de las incorporaciones generadas...

In [ ]:
embed_file = os.path.join(output_dir, 'emb-00000-of-00001.tfrecords')
sample = 5

# Create a description of the features.
feature_description = {
    'text': tf.io.FixedLenFeature([], tf.string),
    'embedding': tf.io.FixedLenFeature([projected_dim], tf.float32)
}

def _parse_example(example):
  # Parse the input `tf.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example, feature_description)

dataset = tf.data.TFRecordDataset(embed_file)
for record in dataset.take(sample).map(_parse_example):
  print("{}: {}".format(record['text'].numpy().decode('utf-8'), record['embedding'].numpy()[:10]))


## 3. Generar el índice ANN para las incorporaciones

[ANNOY](https://github.com/spotify/annoy) (Vecinos más cercanos aproximados, claro que sí) es una biblioteca de C++ con enlaces de Python para buscar puntos en el espacio que estén cerca de un punto de consulta determinado. También crea grandes estructuras de datos basadas en archivos de solo lectura que se asignan a la memoria. [Spotify](https://www.spotify.com) lo creó y lo usa para recomendaciones de música. Si le interesa, puede probar con algunas alternativas de ANNOY como [NGT](https://github.com/yahoojapan/NGT), [FAISS](https://github.com/facebookresearch/faiss), etc. 

In [ ]:
def build_index(embedding_files_pattern, index_filename, vector_length, 
    metric='angular', num_trees=100):
  '''Builds an ANNOY index'''

  annoy_index = annoy.AnnoyIndex(vector_length, metric=metric)
  # Mapping between the item and its identifier in the index
  mapping = {}

  embed_files = tf.io.gfile.glob(embedding_files_pattern)
  num_files = len(embed_files)
  print('Found {} embedding file(s).'.format(num_files))

  item_counter = 0
  for i, embed_file in enumerate(embed_files):
    print('Loading embeddings in file {} of {}...'.format(i+1, num_files))
    dataset = tf.data.TFRecordDataset(embed_file)
    for record in dataset.map(_parse_example):
      text = record['text'].numpy().decode("utf-8")
      embedding = record['embedding'].numpy()
      mapping[item_counter] = text
      annoy_index.add_item(item_counter, embedding)
      item_counter += 1
      if item_counter % 100000 == 0:
        print('{} items loaded to the index'.format(item_counter))

  print('A total of {} items added to the index'.format(item_counter))

  print('Building the index with {} trees...'.format(num_trees))
  annoy_index.build(n_trees=num_trees)
  print('Index is successfully built.')
  
  print('Saving index to disk...')
  annoy_index.save(index_filename)
  print('Index is saved to disk.')
  print("Index file size: {} GB".format(
    round(os.path.getsize(index_filename) / float(1024 ** 3), 2)))
  annoy_index.unload()

  print('Saving mapping to disk...')
  with open(index_filename + '.mapping', 'wb') as handle:
    pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print('Mapping is saved to disk.')
  print("Mapping file size: {} MB".format(
    round(os.path.getsize(index_filename + '.mapping') / float(1024 ** 2), 2)))

In [ ]:
embedding_files = "{}/emb-*.tfrecords".format(output_dir)
embedding_dimension = projected_dim
index_filename = "index"

!rm {index_filename}
!rm {index_filename}.mapping

%time build_index(embedding_files, index_filename, embedding_dimension)

In [ ]:
!ls

## 4. Usar el índice para la coincidencia de similitudes

Ahora podemos usar el índice ANN para encontrar titulares de noticias que estén semánticamente cerca de una consulta de entrada.

### Cargar el índice y los archivos de asignación.

In [ ]:
index = annoy.AnnoyIndex(embedding_dimension)
index.load(index_filename, prefault=True)
print('Annoy index is loaded.')
with open(index_filename + '.mapping', 'rb') as handle:
  mapping = pickle.load(handle)
print('Mapping file is loaded.')


### Método de coincidencia de similitudes

In [ ]:
def find_similar_items(embedding, num_matches=5):
  '''Finds similar items to a given embedding in the ANN index'''
  ids = index.get_nns_by_vector(
  embedding, num_matches, search_k=-1, include_distances=False)
  items = [mapping[i] for i in ids]
  return items

### Extraer la incorporación de una consulta determinada

In [ ]:
# Load the TF-Hub model
print("Loading the TF-Hub model...")
%time embed_fn = hub.load(model_url)
print("TF-Hub model is loaded.")

random_projection_matrix = None
if os.path.exists('random_projection_matrix'):
  print("Loading random projection matrix...")
  with open('random_projection_matrix', 'rb') as handle:
    random_projection_matrix = pickle.load(handle)
  print('random projection matrix is loaded.')

def extract_embeddings(query):
  '''Generates the embedding for the query'''
  query_embedding =  embed_fn([query])[0].numpy()
  if random_projection_matrix is not None:
    query_embedding = query_embedding.dot(random_projection_matrix)
  return query_embedding


In [ ]:
extract_embeddings("Hello Machine Learning!")[:10]

### Ingresar una consulta para encontrar los elementos más similares

In [ ]:
#@title { run: "auto" }
query = "confronting global challenges" #@param {type:"string"}

print("Generating embedding for the query...")
%time query_embedding = extract_embeddings(query)

print("")
print("Finding relevant items in the index...")
%time items = find_similar_items(query_embedding, 10)

print("")
print("Results:")
print("=========")
for item in items:
  print(item)

## ¿Más información?

Puede obtener más información sobre TensorFlow en [tensorflow.org](https://www.tensorflow.org/) y consultar la documentación de la API de TF-Hub en [tensorflow.org/hub](https://www.tensorflow.org/hub/). Encuentre los modelos de TensorFlow Hub disponibles en [tfhub.dev](https://tfhub.dev/), incluidos más modelos de incorporación de texto y modelos de vectores de características de imágenes.

Consulte también el [curso intensivo de aprendizaje automático](https://developers.google.com/machine-learning/crash-course/), que es la introducción práctica y rápida al aprendizaje automático de Google.